In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [7]:
# Define the desired chunk size (25000 rows)
chunk_size = 25000

# Initialize a variable to store the last chunk
last_chunk = None

# Read the CSV file in chunks
chunks = pd.read_csv('Eartquakes-1990-2023.csv', chunksize=chunk_size)

# Process each chunk
for i, chunk in enumerate(chunks):
    # Do whatever processing you need with each chunk
    print(f"Processing chunk {i+1}")
    # Update the last chunk
    last_chunk = chunk

# Save the last chunk to a separate CSV file
last_chunk.to_csv('last_chunk.csv', index=False)


Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Processing chunk 16
Processing chunk 17
Processing chunk 18
Processing chunk 19
Processing chunk 20
Processing chunk 21
Processing chunk 22
Processing chunk 23
Processing chunk 24
Processing chunk 25
Processing chunk 26
Processing chunk 27
Processing chunk 28
Processing chunk 29
Processing chunk 30
Processing chunk 31
Processing chunk 32
Processing chunk 33
Processing chunk 34
Processing chunk 35
Processing chunk 36
Processing chunk 37
Processing chunk 38
Processing chunk 39
Processing chunk 40
Processing chunk 41
Processing chunk 42
Processing chunk 43
Processing chunk 44
Processing chunk 45
Processing chunk 46
Processing chunk 47
Processing chunk 48
Processing chunk 49
Processing chunk 50
Processin

In [8]:
# Read data from file
df = pd.read_csv('last_chunk.csv')

# Drop rows with null values
df_cleaned = df.dropna()

# Select the last 25000 rows
num_rows_to_select = min(25000, len(df_cleaned))
df_last_25000 = df_cleaned.tail(num_rows_to_select)

# Function to extract year, month, day, and hour
def extract_date_components(date_str):
    date_time_obj = pd.to_datetime(date_str)
    return date_time_obj.year, date_time_obj.month, date_time_obj.day, date_time_obj.hour

# Extracting specific columns
df_subset = df_last_25000[['longitude', 'latitude', 'significance', 'magnitudo', 'date']]

# Extracting year, month, day, and hour
df_subset[['year', 'month', 'day', 'hour']] = df_subset['date'].apply(lambda x: pd.Series(extract_date_components(x)))

# Dropping the original 'date' column
df_subset.drop(columns=['date'], inplace=True)

# Save df_subset to a new CSV file
df_subset.to_csv('new_file_last_25000.csv', index=False)

print("Data saved to new_file_last_25000.csv")


Data saved to new_file_last_25000.csv


C:\Users\youss\AppData\Local\Temp\ipykernel_21468\1918219662.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[['year', 'month', 'day', 'hour']] = df_subset['date'].apply(lambda x: pd.Series(extract_date_components(x)))
C:\Users\youss\AppData\Local\Temp\ipykernel_21468\1918219662.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset[['year', 'month', 'day', 'hour']] = df_subset['date'].apply(lambda x: pd.Series(extract_date_components(x)))
C:\Users\youss\AppData\Local\Temp\ipykernel_21

In [9]:
data = np.loadtxt('new_file_last_25000.csv', delimiter=',',skiprows=1)

# # Drop the first row
# data_array = data_array[1:]

print(data)
print(f"The shape(dimensions of the data: {data.shape})")

[[-122.7786667   38.8116667    2.        ...    6.           3.
    14.       ]
 [-122.7778333   38.806        3.        ...    6.           3.
    14.       ]
 [-122.7786636   38.8043327    1.        ...    6.           3.
    14.       ]
 ...
 [-115.2968333   32.2331667   90.        ...    7.          29.
    10.       ]
 [-122.800499    38.8274994   16.        ...    7.          29.
    10.       ]
 [-155.051       57.5648      12.        ...    7.          29.
    11.       ]]
The shape(dimensions of the data: (20751, 8))


In [10]:
# Extracting features (longitude, latitude, year, month, day, hour)
X = data[:, [0, 1, 4, 5, 6, 7]]

# Extracting target variable (magnitudo)
y = data[:, [3]]

print("X shape: ", X.shape)
print("y shape: ", y.shape)



X shape:  (20751, 6)
y shape:  (20751, 1)


In [11]:
## STEP 2

x_train, x_, y_train, y_ = train_test_split(X, y, test_size=0.40, random_state=1)
x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)

del x_, y_

print("X train shape: ",x_train.shape)
print("y train shape: ",y_train.shape)

print("X CV shape:",x_cv.shape)
print("X CV shape:",y_cv.shape)

print("X test shape:",x_test.shape)
print("X test shape:",y_test.shape)

X train shape:  (12450, 6)
y train shape:  (12450, 1)
X CV shape: (4150, 6)
X CV shape: (4150, 1)
X test shape: (4151, 6)
X test shape: (4151, 1)


## Step 3 
## Preprocessing

In [12]:
# Add Polynomial feature
poly = PolynomialFeatures(degree = 3, include_bias=False)
X_train_mapped = poly.fit_transform(x_train)
X_cv_mapped = poly.transform(x_cv)
X_test_mapped = poly.transform(x_test)


In [13]:
# Scale the features using the z-score
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train_mapped)
X_cv_mapped_scaled = scaler.transform(X_cv_mapped)
X_test_mapped_scaled = scaler.transform(X_test_mapped)

## Step 4
## Train and evaluate

In [14]:
# Train the model
model = LinearRegression()
model.fit(X_train_mapped_scaled,y_train)

# Evaluate the model
yhat = model.predict(X_train_mapped_scaled)
print("Training error: ", mean_squared_error(y_train,yhat) / 2)

yhat = model.predict(X_cv_mapped_scaled)
print("Cross-Validation error: ", mean_squared_error(y_cv,yhat) / 2)


Training error:  0.32372428047031626
Cross-Validation error:  0.32331802437831136


<img src='images/C2_W3_poly.png' width=50%>


In [15]:
train_mses = []
cv_mses = []
models = []
polys = []
scalers = []

for degree in range(1,7):
    
    # Add polynomial features to the training set
    poly = PolynomialFeatures(degree, include_bias=False)
    X_train_mapped = poly.fit_transform(x_train)
    polys.append(poly)
    
    # Scale the training set
    scaler_poly = StandardScaler()
    X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)
    scalers.append(scaler_poly)
    
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train_mapped_scaled, y_train )
    models.append(model)
    
    # Compute the training MSE
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    train_mses.append(train_mse)
    
    # Add polynomial features and scale the cross validation set
    X_cv_mapped = poly.transform(x_cv)
    X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)
    
    # Compute the cross validation MSE
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    cv_mses.append(cv_mse)



In [16]:
degree = np.argmin(cv_mses) + 1
print(f"Lowest CV MSE is found in the model with degree= {degree}")

Lowest CV MSE is found in the model with degree= 4


In [17]:
# Add polynomial features to the test set
X_test_mapped = polys[degree-1].transform(x_test)

# Scale the test set
X_test_mapped_scaled = scalers[degree-1].transform(X_test_mapped)

# Compute the test MSE
yhat = models[degree-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"Training MSE: {train_mses[degree-1]:.2f}")
print(f"Cross Validation MSE: {cv_mses[degree-1]:.2f}")
print(f"Test MSE: {test_mse:.2f}")

Training MSE: 0.31
Cross Validation MSE: 0.30
Test MSE: 0.31


## Neural Networks

The same model selection process can also be used when choosing between different neural network architectures. In this section, you will create the models shown below and apply it to the same regression task above.

<img src='images\C2_W3_NN_Arch.png' width=40%>

In [18]:
# Add polynomial features
degree = 1
poly = PolynomialFeatures(degree, include_bias=False)
X_train_mapped = poly.fit_transform(x_train)
X_cv_mapped = poly.transform(x_cv)
X_test_mapped = poly.transform(x_test)

In [19]:
# Scale the features using the z-score
scaler = StandardScaler()
X_train_mapped_scaled = scaler.fit_transform(X_train_mapped)
X_cv_mapped_scaled = scaler.transform(X_cv_mapped)
X_test_mapped_scaled = scaler.transform(X_test_mapped)

In [20]:
def build_models():
    
    tf.random.set_seed(20)
    
    model_1 = Sequential(
        [
            Dense(25, activation = 'relu'),
            Dense(15, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_1'
    )

    model_2 = Sequential(
        [
            Dense(20, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(20, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_2'
    )

    model_3 = Sequential(
        [
            Dense(32, activation = 'relu'),
            Dense(16, activation = 'relu'),
            Dense(8, activation = 'relu'),
            Dense(4, activation = 'relu'),
            Dense(12, activation = 'relu'),
            Dense(1, activation = 'linear')
        ],
        name='model_3'
    )
    
    model_list = [model_1, model_2, model_3]
    
    return model_list

In [34]:
nn_train_mses = []
nn_cv_mses = []

nn_models = build_models()

for model in nn_models:

    # Setup the loss and optimizer
    model.compile(
    loss='mse',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    )


    print(f"Training {model.name}...")
    
    # Train the model
    model.fit(
        X_train_mapped_scaled, y_train,
        epochs=500,
        verbose=0
    )
    
    print("Done!\n")

    # Record the training MSEs
    yhat = model.predict(X_train_mapped_scaled)
    train_mse = mean_squared_error(y_train, yhat) / 2
    nn_train_mses.append(train_mse)
    
    # Record the cross validation MSEs 
    yhat = model.predict(X_cv_mapped_scaled)
    cv_mse = mean_squared_error(y_cv, yhat) / 2
    nn_cv_mses.append(cv_mse)

    
# print results
print("RESULTS:")
for model_num in range(len(nn_train_mses)):
    print(
        f"Model {model_num+1}: Training MSE: {nn_train_mses[model_num]:.2f}, " +
        f"CV MSE: {nn_cv_mses[model_num]:.2f}"
        )


Training model_1...
Done!

390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step
Training model_2...
Done!

390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step
Training model_3...
Done!

390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step
RESULTS:
Model 1: Training MSE: 0.37, CV MSE: 0.37
Model 2: Training MSE: 0.74, CV MSE: 0.74
Model 3: Training MSE: 0.74, CV MSE: 0.74


In [35]:
# Select the model with the lowest CV MSE
model_num = 1

# Compute the test MSE
yhat = nn_models[model_num-1].predict(X_test_mapped_scaled)
test_mse = mean_squared_error(y_test, yhat) / 2

print(f"Selected Model: {model_num}")
print(f"Training MSE: {nn_train_mses[model_num-1]:.2f}")
print(f"Cross Validation MSE: {nn_cv_mses[model_num-1]:.2f}")
print(f"Test MSE: {test_mse:.2f}")

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Selected Model: 1
Training MSE: 0.37
Cross Validation MSE: 0.37
Test MSE: 0.38
